# Add a new plugin

By default, the library will import all the files with prefix "plugin\_\*.py" from src/synthcity/plugins, and load all the classes which implement the [Plugin interface](src/synthcity/plugins/core/plugin.py).

Each plugin must implement the following methods:
- hyperparameter_space() - a static method that returns the hyperparameters that can be tuned during AutoML.
- type() - a static method that returns the type of the plugin. e.g., debug, generative, bayesian, etc.
- name() - a static method that returns the name of the plugin. e.g., ctgan, random_noisee, etc.
- _fit() - internal method, called by `fit` on each training set.
- _generate() - internal method, called by `generate`.

## Existing plugins

In [ ]:
from synthcity.plugins import Plugins

generators = Plugins()

generators.list()

## Example plugin: Generate 0-1

In [ ]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema
from synthcity.plugins.core.dataloader import GenericDataLoader, DataLoader


class ZeroOnePlugin(Plugin):
    """Dummy plugin for debugging."""

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "zero_one"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: DataLoader, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        self.features_count = X.shape[1]
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        return GenericDataLoader(
            np.random.randint(0, 2, size=(count, self.features_count))
        )

In [ ]:
# Add the new plugin to the collection

generators.add("zero_one", ZeroOnePlugin)

In [ ]:
# Check the new plugins list
generators.list()

In [ ]:
# Load reference data

from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True, as_frame=True)

loader = GenericDataLoader(X)

loader.dataframe()

In [ ]:
# Train the new plugin

gen = generators.get("zero_one")

gen.fit(loader)

In [ ]:
# Generate some new data

gen.generate(count=10)

### Oops, this didn't work.

__The Plugin interface enforces the new generated data to:__
 - satistify the same constraints as the training set.
 - Or to satisfy the constraints provided at inference time(if provided).
 
 
 If the generated dataframe fails to comply, an exception will be raised.

Let's try again

## A functional plugin

In [ ]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class DummyGeneratorPlugin(Plugin):
    """Dummy plugin for debugging."""

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "dummy_generator"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: DataLoader, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        result = self.schema().sample(count)
        result[syn_schema.features()] = syn_schema.sample(count)

        return GenericDataLoader(result)

In [ ]:
generators.add("dummy_generator", DummyGeneratorPlugin)

generators.list()

In [ ]:
# Train the new plugin

gen = generators.get("dummy_generator")

gen.fit(loader)

In [ ]:
# Generate some new data

gen.generate(count=10).dataframe()

In [ ]:
# Custom generation constraints

from synthcity.plugins.core.constraints import Constraints

constraints = Constraints(rules=[("worst radius", ">", 15)])

generated = gen.generate(count=10, constraints=constraints)

assert (generated["worst radius"] > 15).any()

generated.dataframe()